Step 1: Install libraries

In [5]:
!pip install transformers accelerate torch


Step 2: Load Mistral Model

In [6]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cpu


Step 3: Prompt + Safety Logic

In [7]:
def is_unsafe_query(query):
    unsafe_keywords = [
        "diagnose", "dosage", "dose", "prescription",
        "treatment", "cure", "how much"
    ]
    return any(word in query.lower() for word in unsafe_keywords)


def health_chatbot(user_query):

    if is_unsafe_query(user_query):
        return (
            "I can share general health information, "
            "but for diagnosis or treatment, please consult "
            "a qualified healthcare professional."
        )

    prompt = f"""
You are a helpful and friendly medical assistant.
You provide general health information only.
You do not diagnose diseases or prescribe medicines.

User question: {user_query}
Answer:
"""

    output = generator(
        prompt,
        max_new_tokens=150,
        temperature=0.5,
        do_sample=True
    )

    return output[0]["generated_text"].split("Answer:")[-1].strip()


Step 4: Test

In [ ]:
print(health_chatbot("What causes a sore throat?"))
print(health_chatbot("Is paracetamol safe for children?"))
print(health_chatbot("What dosage should I take for fever?"))
